<a href="https://colab.research.google.com/github/tashir0605/Cocepts-And-Practice/blob/main/Deep_Learning/DataAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Augmentation – Summary**

## **1. Concept**
- **Purpose:** A technique to increase the amount and diversity of training data without actually collecting new samples.  
- **Benefit:** Helps deep learning models perform better, especially when available data is limited.  
- **Idea:** Create variations of existing data (e.g., flipping, rotating, adding noise) so the model learns to recognize patterns in different conditions.  
- **Advanced Approach:** Adaptive augmentation changes transformations based on the model’s learning progress, making training more effective.  

---

## **2. Augmentation Techniques**

### **A. Audio Data Augmentation**
1. **Noise Injection** – Add Gaussian or random noise to make the model robust to background sounds.  
2. **Shifting** – Shift the audio forward or backward by random time intervals.  
3. **Speed Change** – Stretch or compress the audio timeline at a fixed rate.  
4. **Pitch Change** – Randomly adjust the pitch of the audio.  

### **B. Text Data Augmentation**
1. **Word/Sentence Shuffling** – Randomly reorder words or sentences.  
2. **Word Replacement** – Swap words with their synonyms.  
3. **Syntax-Tree Manipulation** – Paraphrase while keeping the meaning intact.  
4. **Random Word Insertion** – Add extra words in random positions.  
5. **Random Word Deletion** – Remove words at random to encourage model robustness.  


# **Data Augmentation in PyTorch: Cutout & Mixup**

---

## **1. Cutout Augmentation**

**Concept:**  
Cutout randomly masks out square regions of the input image, forcing the model to rely on less obvious parts of the object for classification.  
This improves robustness by preventing over-reliance on specific features.

---

### **Code: Cutout with Albumentations**
```python
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define a transformation pipeline
transforms_cutout = A.Compose([
    A.Resize(256, 256),  # Resize all images to 256x256 pixels
    A.CoarseDropout(
        max_holes=1,        # Number of regions to cut out (1 in this example)
        max_height=128,     # Max height of cutout region
        max_width=128,      # Max width of cutout region
        fill_value=0,       # Pixel value to fill the cutout region (black)
        p=0.5               # Probability of applying Cutout to an image
    ),
    ToTensorV2(),          # Convert NumPy image to PyTorch tensor
])


## **Cutout Pipeline — Step-by-Step**

1. **`A.Resize(256, 256)`** → Ensures all images are the same size.  
2. **`A.CoarseDropout(...)`** → Randomly removes a square patch from the image.  
   - **`fill_value=0`** → The removed patch is filled with black pixels.  
   - **`p=0.5`** → Only half of the images will have a cutout applied.  
3. **`ToTensorV2()`** → Converts the image to a **PyTorch tensor** (required for model input).

# **2. Mixup Augmentation**

---

## **Concept**
Mixup creates new training samples by **blending two random images and their labels**.  
This encourages the model to behave more **linearly between samples** and helps **prevent overfitting**.

---

## **Code: Mixup Implementation**
```python
import torch
import numpy as np
import torch.nn as nn

# Function to apply Mixup to a batch
def mixup(data, targets, alpha):
    indices = torch.randperm(data.size(0))   # Randomly shuffle the batch
    shuffled_data = data[indices]            # Get shuffled images
    shuffled_targets = targets[indices]      # Get shuffled labels

    lam = np.random.beta(alpha, alpha)       # Mixing factor from Beta distribution
    new_data = data * lam + shuffled_data * (1 - lam)  # Blend images
    new_targets = [targets, shuffled_targets, lam]     # Store both sets of labels and lam
    return new_data, new_targets

# Custom loss function for Mixup
def mixup_criterion(preds, targets):
    targets1, targets2, lam = targets
    criterion = nn.CrossEntropyLoss()
    return lam * criterion(preds, targets1) + (1 - lam) * criterion(preds, targets2)


## **Step-by-step Explanation (Mixup Function)**

1. **`torch.randperm(data.size(0))`** → Creates a random permutation of the batch indices.  
2. **`shuffled_data` / `shuffled_targets`** → The original batch reordered randomly using those indices.  
3. **`lam`** → A random blending factor sampled from a **Beta** distribution (`np.random.beta(alpha, alpha)`).  
4. **`new_data`** → Weighted combination of original and shuffled images: `data * lam + shuffled_data * (1 - lam)`.  
5. **`new_targets`** → A container holding both label sets and the mixing weight: `[targets, shuffled_targets, lam]`.  
6. **`mixup_criterion`** → Computes the weighted loss for both labels:  
   `lam * CE(preds, targets1) + (1 - lam) * CE(preds, targets2)`.


**Using Mixup in the Training Loop**


In [ ]:
p_mixup = 0.5  # Probability of applying Mixup

for epoch in range(NUM_EPOCHS):
    model.train()  # Set model to training mode

    for samples, labels in train_dataloader:
        samples, labels = samples.to(device), labels.to(device)

        samples = samples / 255  # Normalize pixel values (0-255 → 0-1)

        # Decide whether to apply Mixup
        p = np.random.rand()
        if p < p_mixup:
            samples, labels = mixup(samples, labels, alpha=0.8)

        optimizer.zero_grad()  # Reset gradients

        # Forward pass
        output = model(samples)

        # Compute loss (with Mixup or normal)
        if p < p_mixup:
            loss = mixup_criterion(output, labels)
        else:
            loss = nn.CrossEntropyLoss()(output, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()


## **Step-by-step Explanation (Training Loop)**

1. **`p_mixup`** → Sets the probability of applying Mixup augmentation (e.g., 0.5 = 50% of batches).  
2. **Normalize images** → Scales pixel values from `[0, 255]` to `[0, 1]` for better training stability.  
3. **Mixup decision** → Randomly decide for each batch whether Mixup should be applied.  
4. **Forward pass** → Feed the batch through the model to get predictions (`output = model(samples)`).  
5. **Loss calculation** →  
   - **If Mixup applied** → Use `mixup_criterion` to compute the weighted loss for both label sets.  
   - **If Mixup not applied** → Use the standard `CrossEntropyLoss`.  
6. **Backpropagation & optimizer step** → Compute gradients with `.backward()` and update model weights using `optimizer.step()`.  


# **CutMix Augmentation**

---

## **Concept**
CutMix is similar to Mixup, but instead of blending entire images,  
it **cuts a rectangular patch from one image** and pastes it into another,  
adjusting the labels according to the area replaced.  
This helps the model learn from **local features** as well as global context.

---

## **Code: CutMix Implementation**
```python
import torch
import numpy as np

# Function to apply CutMix to a batch
def cutmix(data, targets, alpha):
    # Shuffle the batch
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    # Mixing factor
    lam = np.random.beta(alpha, alpha)

    # Get bounding box coordinates for the cut
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)

    # Replace the patch in original images with the patch from shuffled images
    data[:, :, bbx1:bbx2, bby1:bby2] = data[indices, :, bbx1:bbx2, bby1:bby2]

    # Adjust lambda to match the exact area ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size()[-1] * data.size()[-2]))

    # Store both labels and lambda
    new_targets = [targets, shuffled_targets, lam]
    return data, new_targets

# Helper function to generate random bounding box
def rand_bbox(size, lam):
    W = size[2]  # Image width
    H = size[3]  # Image height
    cut_rat = np.sqrt(1. - lam)  # Cut ratio based on lambda
    cut_w = int(W * cut_rat)     # Cut width
    cut_h = int(H * cut_rat)     # Cut height

    # Random center point for the cut
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    # Bounding box coordinates, clipped to image boundaries
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2


# **Step-by-Step Explanation – CutMix Function**

1. **Shuffle Batch Indices**  
   - `torch.randperm(data.size(0))` creates a random permutation of batch indices.  
   - This ensures images and labels are paired with different samples.

2. **Get Shuffled Data & Labels**  
   - `shuffled_data` = images from shuffled batch.  
   - `shuffled_targets` = labels from shuffled batch.

3. **Sample Mixing Factor**  
   - `lam` is drawn from a Beta distribution.  
   - Controls how much of each image is kept.

4. **Generate Bounding Box**  
   - Call `rand_bbox()` to get a random rectangular region to cut and replace.

5. **Replace Image Patch**  
   - In `data`, replace the selected patch with the corresponding patch from `shuffled_data`.

6. **Adjust Lambda Value**  
   - Recalculate `lam` based on **actual pixel area replaced** for accuracy.

7. **Return Modified Data & Labels**  
   - Output:  
     - Modified `data` (images).  
     - `new_targets` = `[original_labels, shuffled_labels, lam]`.


# **Step-by-Step Explanation – rand_bbox Function**

1. **Extract Image Dimensions**  
   - `W` = image width, `H` = image height.

2. **Compute Cut Ratio**  
   - `cut_rat = sqrt(1 - lam)` determines the size of the cut.

3. **Calculate Cut Dimensions**  
   - `cut_w` = width of patch.  
   - `cut_h` = height of patch.

4. **Pick Random Center Point**  
   - `(cx, cy)` = center coordinates of the cut patch.

5. **Clip Coordinates to Image Bounds**  
   - Ensure `(bbx1, bby1, bbx2, bby2)` stay within image dimensions.

6. **Return Bounding Box Coordinates**  
   - Output: `(bbx1, bby1, bbx2, bby2)`.


The rest is the same as for Mixup:

1.  Define a cutmix_criterion() functions to handle the custom loss (see the implementation of mixup_criterion())

2.  Define a variable p_cutmix to control the portion of batches that will be augmented (see p_mixup)

3.  Apply cutmix() and cutmix_criterion() in accordance to p_cutmix in the training code.